In [1]:
#from urllib.request import urlopen
import json

import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon

import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio

import matplotlib.pyplot as plt

import numpy as np

## Shapes

In [2]:
with open('geojson/continente.geojson', encoding='utf-8') as file:
    continente = json.loads(file.read())
    
with open('geojson/madeira.geojson', encoding='utf-8') as file:
    madeira = json.loads(file.read())

with open('geojson/azores.geojson', encoding='utf-8') as file:
    azores = json.loads(file.read())

In [3]:
continente['features'][0]['properties']

{'distrito': 'Évora', 'concelho': 'Évora', 'CCA_2': '0705'}

In [4]:
# add IDs to regions (features)
for feature in continente["features"]:
    feature['id'] = feature['properties']['CCA_2']
for feature in madeira["features"]:
    feature['id'] = feature['properties']['CCA_2']
for feature in azores["features"]:
    feature['id'] = feature['properties']['CCA_2']

In [5]:
azores['features'][0]['properties']

{'distrito': 'Azores', 'concelho': 'Angra do heroísmo', 'CCA_2': '4301'}

## Data

In [6]:
# generate data
data_list = []

for region in (continente, madeira, azores):
    np.random.seed(len(region['features']))
    
    ids = [i['properties']['CCA_2'] for i in region['features']]
    names = [i['properties']['concelho'] for i in region['features']]
    values = [int(100*np.random.random()) for i in range(len(region['features']))]
    
    data = pd.DataFrame([ids, names, values]).T.rename(columns={0:'id', 1: 'concelho', 2: 'value'})
    data['value'] = data['value'].astype(int)
    data_list.append(data)
    
continente_data, madeira_data, azores_data = data_list

In [28]:
continente_data

,id,concelho,value
0,0705,Évora,100
1,0701,Alandroal,90
2,0702,Arraiolos,51
3,0703,Borba,73
4,0704,Estremoz,60
...,...,...,...
273,1820,Tarouca,59
274,1821,Tondela,9
275,1822,Vila nova de paiva,36
276,1823,Viseu,20


In [24]:
continente_data.iloc[0,2]=100
continente_data.iloc[:,1].tolist()

['Évora',
 'Alandroal',
 'Arraiolos',
 'Borba',
 'Estremoz',
 'Montemor-o-novo',
 'Mora',
 'Mourão',
 'Portel',
 'Redondo',
 'Reguengos de monsaraz',
 'Vendas novas',
 'Viana do alentejo',
 'Vila viçosa',
 'Águeda',
 'Ílhavo',
 'Albergaria-a-velha',
 'Anadia',
 'Arouca',
 'Aveiro',
 'Castelo de paiva',
 'Espinho',
 'Estarreja',
 'Mealhada',
 'Murtosa',
 'Oliveira de azeméis',
 'Oliveira do bairro',
 'Ovar',
 'São joão da madeira',
 'Santa maria da feira',
 'Sever do vouga',
 'Vagos',
 'Vale de cambra',
 'Aljustrel',
 'Almodôvar',
 'Alvito',
 'Barrancos',
 'Beja',
 'Castro verde',
 'Cuba',
 'Ferreira do alentejo',
 'Mértola',
 'Moura',
 'Odemira',
 'Ourique',
 'Serpa',
 'Vidigueira',
 'Amares',
 'Barcelos',
 'Braga',
 'Braga',
 'Cabeceiras de basto',
 'Celorico de basto',
 'Esposende',
 'Fafe',
 'Póvoa de lanhoso',
 'Terras de bouro',
 'Vieira do minho',
 'Vila nova de famalicão',
 'Vila verde',
 'Vizela',
 'Alfândega da fé',
 'Bragança',
 'Carrazeda de ansiães',
 'Freixo de espada à ci

In [16]:
palette = {'cutoff' : 'rgba(227, 78, 38, 1)', 
           'wave1' : 'rgba(247, 141, 31, 0.7)', 
           'wave2' : 'rgba(227, 78, 38, 1)',
           'scale0' : 'rgba(172, 185, 54, 0)',
           'scale1' : 'rgba(0, 120, 128, 1)',
           'scale2' : 'rgba(0, 102, 102, 1)',
           'scale3' : 'rgba(0, 70, 70, 1)',
           'borders' : 'rgba(0, 0, 0, 1)',
           'text' : 'rgba(114, 114, 114, 1)',
           'legendtext' : 'black',
           'bartext': 'black',
           'bartitletext' : 'black',
           'bubbletext' : 'black'
         }

In [17]:
fig = go.Figure()    

## Choropleth map ######
fig.add_choropleth(geojson=continente, locations=continente_data.id,
                        z=continente_data.value,
                        colorscale="teal",
                        zmin=0,
                        zmax=100,
                        hovertext = continente_data.concelho,
                        hoverinfo="text",
                        colorbar=dict(title={'text':'',
                                             'font':{#'size':24,
                                                     'family' : 'Arial',
                                                     'color' : palette['text']},
                                             'side':'right'},
                                      tickfont={#'size' : 20, 
                                                'color' : palette['bartext']},
                                      #ticks='outside',
                                      #len=0.1,
                                      #x=0.1
                                     ),
                        marker = dict(line=dict(width=1))
                        )
fig.update_geos(fitbounds="locations", bgcolor='rgba(0,0,0,0)', visible=False)
fig.show()
#pio.write_image(fig, 'test.png', width=1600, height=800, engine='kaleido', scale=2)